In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load datasets
df_fishing = pd.read_csv('industrial_vessels_v20240102.csv')  # Fishing vessel data
df_whale_risk = pd.read_csv('aggregated_whale_data.csv')  # Whale risk data

# Bin fishing data to match whale risk grid
df_fishing['lat_bin'] = round(df_fishing['lat'], 1)
df_fishing['lon_bin'] = round(df_fishing['lon'], 1)

# Aggregate fishing scores by grid cell
fishing_by_cell = df_fishing.groupby(['lat_bin', 'lon_bin']).agg({
    'fishing_score': 'mean',
    'scene_id': 'count'
}).reset_index()
fishing_by_cell.rename(columns={'scene_id': 'fishing_count'}, inplace=True)

# Merge with whale risk data
merged = pd.merge(
    df_whale_risk,
    fishing_by_cell,
    on=['lat_bin', 'lon_bin'],
    how='left'
)

# Fill NA for cells with no fishing activity
merged['fishing_score'] = merged['fishing_score'].fillna(0)
merged['fishing_count'] = merged['fishing_count'].fillna(0)

# Calculate correlation
correlation = merged[['fishing_score', 'all.risk']].corr().iloc[0,1]

# Create interactive scatter plot
fig1 = px.scatter(
    merged,
    x='fishing_score',
    y='all.risk',
    color='mgmt',
    hover_data=['lat_bin', 'lon_bin', 'fishing_count'],
    title=f'Fishing Activity vs. Whale Risk (Correlation: {correlation:.2f})',
    labels={
        'fishing_score': 'Average Fishing Score',
        'all.risk': 'Composite Whale Risk Score',
        'mgmt': 'Management Zone'
    }
)

# Create map visualization
fig2 = px.scatter_geo(
    merged,
    lat='lat_bin',
    lon='lon_bin',
    color='fishing_score',
    size='all.risk',
    hover_name='mgmt',
    hover_data=['fishing_count', 'all.risk'],
    title='Spatial Distribution of Fishing and Whale Risk',
    projection='natural earth'
)

# Create composite figure
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Fishing vs. Whale Risk', 'Spatial Distribution')
)

fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)

fig.update_layout(height=600, width=1200, showlegend=False)
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'industrial_vessels_v20240102.csv'